In [1]:
from joblib import load
import pandas as pd
from itertools import permutations
import pickle

# list_tuplas es una lista de tuplas compuestas por los días en los que se puede jugar y un 0 o 1 dependiendo de si es finde o no
filename = '../Data/list_tuplas.pkl'
lista_tuplas = open(filename, 'rb')
with open(filename, 'rb') as f:
    lista_tuplas = pickle.load(f)

# Cargamos los datos de los equipos para crear los partidos
# Los partidos están definidos como tuplas de dos equipos, en el que la primera posición la ocupa el local
df = pd.read_csv('../Data/equipos_data.csv', index_col = 0)
partidos = list(permutations(df['equipo'].unique(), 2))

# Cargamos el modelo de predicción de audiencia ya entrenado
model = load('trainedMLP.joblib')

C:\Users\migue\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [2]:
import pandas as pd

data = []

# Creamos un dataframe en el que cada fila será cada combinación posible de partido y día
# Además, incluimos el mes y si es findo o no, ya el modelo fue entrenado con estas variables
for fecha, finde in lista_tuplas:
    for partido in partidos:
        data.append(partido + (int(fecha[3:5]), finde, fecha))
        
df = pd.DataFrame(data, columns = ['equipo1', 'equipo2', 'mes', 'fin_de_semana', 'fecha'])
df.astype({'mes': 'object'}).dtypes

# Dummificamos las variables categóricas
historico = pd.get_dummies(df.drop('fecha', axis = 1), prefix = ['equipo1','equipo2','mes'], columns = ['equipo1','equipo2','mes'])

In [3]:
# columnas.txt es un fichero que tiene almacenadas las columnas con las que se entrenó el modelo
# Cargamos el fichero y almacenamos los nombres en orden_original
orden_original = []

with open('columnas.txt', encoding='utf-8') as fp:
    for fila in fp:
        orden_original.append(fila[:-1])

# Eliminamos la columna de audiencia
orden_original.pop(1)

# Comprobamos que las columnas están en el mismo orden
# De no estarlo, las predicciones no serían correctas
all(historico.columns == orden_original)

True

In [18]:
# Vamos a almacenar las predicciones en un diccionario donde las claves serán
# una tupla de partido y día, y el valor será la audiencia predicha para este
# encuentro en este día
predicciones = {}
df['pred'] = model.predict(historico)

for i, fila in df.iterrows():
    predicciones[(fila['equipo1'], fila['equipo2']), fila['fecha']] = fila['pred']

# Guardamos las predicciones para utilizarlas como parámetros del modelo de optimización
with open('../Data/predicciones.pickle', 'wb') as file:
    pickle.dump(predicciones, file)

C:\Users\migue\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
